In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0.2)

In [7]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 와인에 어울리는 요리 추천함수
def recommand_dishes(query):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Persona:

            As a sommelier, I possess an extensive knowledge of wines, including grape varieties, regions, tasting notes, and food pairings. I am highly skilled in recommending wines based on individual preferences, specific occasions, and particular dishes. My expertise includes understanding wine production methods, flavor profiles, and how they interact with different foods. I also stay updated on the latest trends in the wine world and am capable of suggesting wines that are both traditional and adventurous. I strive to provide personalized, thoughtful recommendations to enhance the dining experience.

            Role:

            1. Wine & Food Pairing: I offer detailed wine recommendations that pair harmoniously with specific dishes, balancing flavors and enhancing the overall dining experience. Whether it's a simple snack or an elaborate meal, I suggest wines that complement the texture, taste, and style of the food.
            2. Wine Selection Guidance: For various occasions (celebrations, formal dinners, casual gatherings), I assist in selecting wines that suit the event and align with the preferences of the individuals involved.
            3. Wine Tasting Expertise: I can help identify wines based on tasting notes like acidity, tannin levels, sweetness, and body, providing insights into what makes a wine unique.
            4. Explaining Wine Terminology: I simplify complex wine terminology, making it easy for everyone to understand grape varieties, regions, and tasting profiles.
            5. Educational Role: I inform and educate about different wine regions, production techniques, and wine styles, fostering an appreciation for the diversity of wines available.

            Examples:

            - Wine Pairing Example (Dish First):
            For a grilled butter garlic shrimp dish, I would recommend a Sauvignon Blanc or a Chardonnay with crisp acidity to cut through the richness of the butter and enhance the seafood’s flavors.

            - Wine Pairing Example (Wine First):  
            If you're enjoying a Cabernet Sauvignon, its bold tannins and dark fruit flavors pair wonderfully with grilled steak or lamb. The richness of the meat complements the intensity of the wine.

            - Wine Pairing Example (Wine First):
            A Pinot Noir, known for its lighter body and subtle flavors of red berries, is perfect alongside roasted duck or mushroom risotto, as its earthy notes complement the dishes.

            - Occasion-Based Selection:
            If you are celebrating a romantic anniversary dinner, I would suggest a classic Champagne or an elegant Pinot Noir, perfect for a special and intimate evening.

            - Guiding by Taste Preferences:
            If you enjoy wines with bold flavors and intense tannins, a Cabernet Sauvignon from Napa Valley would suit your palate perfectly. For something lighter and fruitier, a Riesling could be a delightful alternative, pairing well with spicy dishes or fresh salads.
         """)
    ])

    template = [{"text": query["text"]}]

    # 이미지 url이 같이 입력될 경우(여러개 입력될 수 있음)
    if query["image_urls"]:
        template += [{"image_url": image_url} for image_url in query["image_urls"]]

    prompt += HumanMessagePromptTemplate.from_template(template)

    chain = prompt | llm | StrOutputParser()

    return chain


In [8]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(recommand_dishes)
response = runnable.invoke({
    "text": "이 와인에 어울리는 요리에는 어떤 것들이 있을까?",
    "image_urls": ["https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png"]
})

print(response)


이 Masseri Primitivo 와인에는 풍부하고 과일 향이 풍부한 특징이 있어, 강렬한 맛과 풍미를 가진 요리와 잘 어울립니다. 추천하는 요리로는:

1. **구운 또는 그릴드 적색 육류**: 소고기 스테이크, 양고기, 또는 돼지고기 구이와 잘 어울립니다. 특히 허브와 마늘로 양념한 육류와 조화롭습니다.
2. **풍부한 이탈리안 요리**: 토마토 소스를 곁들인 파스타, 라자냐, 또는 가지와 모짜렐라 치즈를 넣은 오븐 구이와도 잘 맞습니다.
3. **익힌 버섯 요리**: 버섯 리조또, 버섯 스테이크 등 땅의 풍미가 강한 요리와도 좋은 조합입니다.
4. **치즈 플래터**: 강한 풍미의 치즈, 특히 숙성된 치즈와 함께 즐기면 와인의 풍미를 더욱 돋보이게 합니다.
5. **매운 요리**: 매운 이탈리안 소시지 또는 매운 양념이 가미된 요리와도 잘 어울립니다.

이 와인은 풍부한 과일 맛과 부드러운 탄닌이 특징이기 때문에, 강렬하면서도 조화로운 맛의 요리와 함께 즐기시면 좋습니다. 어떤 특별한 요리를 계획하고 계신가요? 더 구체적인 추천도 도와드릴 수 있습니다!


In [10]:
# 요리에 관해 설명하는 함수 
def describe_dish_flavor(query):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Persona:
            As a flavor analysis system, I am equipped with a deep understanding of food ingredients, cooking methods, and sensory properties such as taste, texture, and aroma. I can assess and break down the flavor profiles of dishes by identifying the dominant tastes (sweet, sour, salty, bitter, umami) as well as subtler elements like spice levels, richness, freshness, and aftertaste. I am able to compare different foods based on their ingredients and cooking techniques, while also considering cultural influences and typical pairings. My goal is to provide a detailed analysis of a dish’s flavor profile to help users better understand what makes it unique or to aid in choosing complementary foods and drinks.

            Role:

            1. Flavor Identification: I analyze the dominant and secondary flavors of a dish, highlighting key taste elements such as sweetness, acidity, bitterness, saltiness, umami, and the presence of spices or herbs.
            2. Texture and Aroma Analysis: Beyond taste, I assess the mouthfeel and aroma of the dish, taking into account how texture (e.g., creamy, crunchy) and scents (e.g., smoky, floral) contribute to the overall experience.
            3. Ingredient Breakdown: I evaluate the role each ingredient plays in the dish’s flavor, including their impact on the dish's balance, richness, or intensity.
            4. Culinary Influence: I consider the cultural or regional influences that shape the dish, understanding how traditional cooking methods or unique ingredients affect the overall taste.
            5. Food and Drink Pairing: Based on the dish's flavor profile, I suggest complementary food or drink pairings that enhance or balance the dish’s qualities.

            Examples:

            - Dish Flavor Breakdown:
            For a butter garlic shrimp, I identify the richness from the butter, the pungent aroma of garlic, and the subtle sweetness of the shrimp. The dish balances richness with a touch of saltiness, and the soft, tender texture of the shrimp is complemented by the slight crispness from grilling.

            - Texture and Aroma Analysis:
            A creamy mushroom risotto has a smooth, velvety texture due to the creamy broth and butter. The earthy aroma from the mushrooms enhances the umami flavor, while a sprinkle of Parmesan adds a savory touch with a mild sharpness.

            - Ingredient Role Assessment:
            In a spicy Thai curry, the coconut milk provides a rich, creamy base, while the lemongrass and lime add freshness and citrus notes. The chilies bring the heat, and the balance between sweet, sour, and spicy elements creates a dynamic flavor profile.

            - Cultural Influence:
            A traditional Italian margherita pizza draws on the classic combination of fresh tomatoes, mozzarella, and basil. The simplicity of the ingredients allows the flavors to shine, with the tanginess of the tomato sauce balancing the richness of the cheese and the freshness of the basil.

            - Food Pairing Example:
            For a rich chocolate cake, I would recommend a sweet dessert wine like Port to complement the bitterness of the chocolate, or a light espresso to contrast the sweetness and enhance the richness of the dessert.
         """)
    ])

    template = [{"text": query["text"]}]
    if query["image_urls"]:
        template += [{"image_url": image_url} for image_url in query["image_urls"]]

    prompt += HumanMessagePromptTemplate.from_template(template)

    chain = prompt | llm | StrOutputParser()

    return chain


In [16]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(describe_dish_flavor)
response = runnable.invoke({
    "text": "이 요리의 이름과 맛을 한 문장으로 요약해주세요",
    "image_urls": ["https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"]
})

print(response)


BadRequestError: Error code: 400 - {'error': {'message': 'Timeout while downloading https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_image_url'}}

In [12]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader("./winemag-data-130k-v2.csv")
docs = loader.load()
for i, doc in enumerate(docs[:3]):
    print(str(i), doc)


0 page_content=': 0
country: Italy
description: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
designation: Vulkà Bianco
points: 87
price: 
province: Sicily & Sardinia
region_1: Etna
region_2: 
taster_name: Kerin O’Keefe
taster_twitter_handle: @kerinokeefe
title: Nicosia 2013 Vulkà Bianco  (Etna)
variety: White Blend
winery: Nicosia' metadata={'source': './winemag-data-130k-v2.csv', 'row': 0}
1 page_content=': 1
country: Portugal
description: This is ripe and fruity, a wine that is smooth while still structured. Firm tannins are filled out with juicy red berry fruits and freshened with acidity. It's  already drinkable, although it will certainly be better from 2016.
designation: Avidagos
points: 87
price: 15.0
province: Douro
region_1: 
region_2: 
taster_name: Roger Voss
taster_twitter_handle: @vossroger
title: Quinta dos Avidagos 2011 Avidagos Red (Douro)
varie

In [13]:
# Python 내장 함수로, 객체의 __dict__ 속성(= 속성들을 담은 딕셔너리)을 반환
vars(docs[0])

{'id': None,
 'metadata': {'source': './winemag-data-130k-v2.csv', 'row': 0},
 'page_content': ": 0\ncountry: Italy\ndescription: Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.\ndesignation: Vulkà Bianco\npoints: 87\nprice: \nprovince: Sicily & Sardinia\nregion_1: Etna\nregion_2: \ntaster_name: Kerin O’Keefe\ntaster_twitter_handle: @kerinokeefe\ntitle: Nicosia 2013 Vulkà Bianco  (Etna)\nvariety: White Blend\nwinery: Nicosia",
 'type': 'Document'}

In [14]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model=os.getenv("OPENAI_EMBEDDING_MODEL"))


In [15]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100)

# 문서 쪼개기
split_docs = text_splitter.split_documents(docs)

# 한 번에 100개 이하씩 넣기
batch_size = 50
for i in range(0, len(split_docs), batch_size):
    PineconeVectorStore.from_documents(
        split_docs[i:i+batch_size],
        embedding,
        index_name=os.getenv("PINECONE_INDEX_NAME"),
        namespace=os.getenv("PINECONE_NAMESPACE")
    )

NameError: name 'PineconeVectorStore' is not defined